# III. Deloy

##  1. Import libraries

In [1]:
import pandas as pd
import joblib
from sklearn.preprocessing import StandardScaler
from google.colab import widgets
from IPython.display import display
import ipywidgets as widgets
import numpy as np


## 2. Import model

In [2]:
model = joblib.load('model/house_rent_model.pkl')
scaler = joblib.load('model/scaler.pkl')

## 3. Create input field

In [3]:

category_options = [
    'Căn hộ chung cư', 'Văn phòng', 'Chung cư mini, căn hộ dịch vụ',
    'Kho, nhà xưởng, đất', 'Shophouse nhà phố thương mại', 'Nhà riêng',
    'Nhà mặt phố', 'Cửa hàng, ki ốt', 'Nhà biệt thự, liền kề',
    'Nhà trọ, phòng trọ'
]

furniture_options = ['Unfurnished', 'Basic', 'Fully Furnished', 'High-end', 'Unspecified']
city_options = ['Thành phố Hồ Chí Minh', 'Hà Nội', 'Bình Dương']

districts_hcm = [
    'quận 1','quận 2', 'quận 3', 'quận 4', 'quận 5', 'quận 6',
    'quận 7', 'quận 8','quận 9', 'quận 10', 'quận 11', 'quận 12',
    'tân bình', 'bình tân', 'bình thạnh',
    'tân phú', 'gò vấp', 'phú nhuận',
    'bình chánh', 'hóc môn', 'cần giờ',
    'củ chi', 'nhà bè', 'thủ đức'
]

districts_hn = [
    'ba đình', 'cầu giấy', 'đống đa', 'hai bà trưng', 'hoàn kiếm',
    'thanh xuân', 'hoàng mai', 'long biên', 'hà đông', 'tây hồ',
    'nam từ liêm', 'bắc từ liêm',
    'thanh trì', 'ba vì', 'đan phượng', 'gia lâm', 'đông anh',
    'thường tín', 'thanh oai', 'chương mỹ', 'hoài đức', 'mỹ đức',
    'phúc thọ', 'thạch thất', 'quốc oai', 'phú xuyên', 'ứng hòa',
    'mê linh', 'sóc sơn', 'sơn tây'
]

districts_bd = [
    'Thuận an', 'tân uyên', 'dĩ an', 'bến cát', 'thủ dầu một',
    'dầu tiếng', 'bàu bàng', 'phú giáo', 'bắc tân uyên'
]



In [4]:
def get_district_options(city):
    if city == 'Thành phố Hồ Chí Minh':
        return districts_hcm
    elif city == 'Hà Nội':
        return districts_hn
    elif city == 'Bình Dương':
        return districts_bd
    return []

In [5]:
category_widget = widgets.Dropdown(options=category_options, description='Category:')
furniture_widget = widgets.Dropdown(options=furniture_options, description='Furniture:')
city_widget = widgets.Dropdown(options=city_options, description='City:')
district_widget = widgets.Dropdown(options=[], description='District:')
area_widget = widgets.FloatText(value=0, description='Area:')
bedrooms_widget = widgets.IntText(value=0, description='Bedrooms:')
toilets_widget = widgets.IntText(value=0, description='Toilets:')
rooms_widget = widgets.IntText(value=0, description='Rooms:')

In [6]:
def update_district_options(change):
    district_widget.options = get_district_options(change['new'])
    district_widget.value = None

city_widget.observe(update_district_options, names='value')

## 4. Predict code

In [14]:
def predict_price(b):
    input_data = {
        'Category': category_widget.value,
        'Furniture': furniture_widget.value,
        'City': city_widget.value,
        'District': district_widget.value.capitalize(),
        'Area': area_widget.value,
        'Number of Bedrooms': bedrooms_widget.value,
        'Number of Toilets': toilets_widget.value,
        'Number of Rooms': rooms_widget.value
    }

    numerical_columns = ['Area', 'Number of Bedrooms', 'Number of Toilets']

    input_df = pd.DataFrame([input_data])

    input_encoded = pd.get_dummies(input_df, columns=['Category', 'Furniture', 'City', 'District'],
                                   prefix=['Category', 'Furniture', 'City', 'District'])

    model_feature_names = model.get_booster().feature_names if hasattr(model, 'get_booster') else model.feature_names_in_

    for col in model_feature_names:
        if col not in input_encoded.columns:
            input_encoded[col] = 0


    input_encoded = input_encoded.reindex(columns=model_feature_names, fill_value=0)


    input_encoded[numerical_columns] = scaler.transform(input_encoded[numerical_columns])

    predicted_price = model.predict(input_encoded)*1_000_000
    print(f"Predicted Price: {predicted_price[0]:,.2f} VND")




## 5. Input Data Here

In [8]:
display(category_widget, furniture_widget, city_widget, district_widget, area_widget, bedrooms_widget, toilets_widget, rooms_widget)



Dropdown(description='Category:', options=('Căn hộ chung cư', 'Văn phòng', 'Chung cư mini, căn hộ dịch vụ', 'K…

Dropdown(description='Furniture:', options=('Unfurnished', 'Basic', 'Fully Furnished', 'High-end', 'Unspecifie…

Dropdown(description='City:', options=('Thành phố Hồ Chí Minh', 'Hà Nội', 'Bình Dương'), value='Thành phố Hồ C…

Dropdown(description='District:', options=(), value=None)

FloatText(value=0.0, description='Area:')

IntText(value=0, description='Bedrooms:')

IntText(value=0, description='Toilets:')

IntText(value=0, description='Rooms:')

In [16]:
predict_button = widgets.Button(description="Predict Price")
predict_button.on_click(predict_price)


display(predict_button)

Button(description='Predict Price', style=ButtonStyle())

Predicted Price: 17,943,514.80 VND
Predicted Price: 5,773,965.12 VND
Predicted Price: 6,826,874.00 VND
Predicted Price: 8,107,024.48 VND
